In [64]:
import json
import requests
import time

# Disable SSL certificate verification
requests.packages.urllib3.disable_warnings()

# Fetch the JSON data from the URL without SSL verification
url = "https://127.0.0.1:2999/liveclientdata/gamestats"

# Make the request and retrieve the data
response = requests.get(url, verify=False)
data = response.json()

# Extract the gameTime value
game_time_seconds = data["gameTime"]

# Convert game_time to minutes and seconds
minutes = game_time_seconds // 60
seconds = game_time_seconds % 60

# Format minutes and seconds without decimal places
formatted_minutes = "{:.0f}".format(minutes)
formatted_seconds = "{:.0f}".format(seconds)

# Print the converted time
print("Game Time: {} minutes {} seconds".format(formatted_minutes, seconds))


Game Time: 24 minutes 57.0595703125 seconds


In [65]:
import requests

playerlistURL = "https://127.0.0.1:2999/liveclientdata/playerlist"
response = requests.get(playerlistURL, verify=False)
data = response.json()

# Calculate the sum of levels for players in "ORDER" team
order_levels = []
for obj in data:
    if obj.get("team") == "ORDER":
        order_levels.append(int(obj["level"]))

# Calculate the sum of levels for players in "CHAOS" team
chaos_levels = []
for obj in data:
    if obj.get("team") == "CHAOS":
        chaos_levels.append(int(obj["level"]))

# Calculate the average levels for each team
order_average = sum(order_levels) / len(order_levels)
chaos_average = sum(chaos_levels) / len(chaos_levels)

# Print the averages
print(f"Average of levels for players in the 'ORDER' team: {order_average}")
print(f"Average of levels for players in the 'CHAOS' team: {chaos_average}")


Average of levels for players in the 'ORDER' team: 14.6
Average of levels for players in the 'CHAOS' team: 12.8


In [66]:
import requests

playerlistURL = "https://127.0.0.1:2999/liveclientdata/playerlist"
response = requests.get(playerlistURL, verify=False)
data = response.json()

# Calculate the sum of scores for players in "ORDER" team
order_scores = {}
for key in data[0]["scores"]:
    order_scores[key] = sum(obj["scores"][key] for obj in data if obj.get("team") == "ORDER")

# Calculate the sum of scores for players in "CHAOS" team
chaos_scores = {}
for key in data[0]["scores"]:
    chaos_scores[key] = sum(obj["scores"][key] for obj in data if obj.get("team") == "CHAOS")

# Print the sums
print("Sum of scores for players in 'ORDER' team:")
print(order_scores)
print("Sum of scores for players in 'CHAOS' team:")
print(chaos_scores)


Sum of scores for players in 'ORDER' team:
{'assists': 40, 'creepScore': 790, 'deaths': 9, 'kills': 34, 'wardScore': 96.9509973526001}
Sum of scores for players in 'CHAOS' team:
{'assists': 12, 'creepScore': 680, 'deaths': 34, 'kills': 9, 'wardScore': 84.47742748260498}


In [67]:
import requests

def collect_items_by_team(json_data):
    team1_items = []
    team2_items = []

    for player in json_data:
        team = player["team"]
        items = player["items"]

        if team == "ORDER":
            team1_items.extend(item["itemID"] for item in items)
        elif team == "CHAOS":
            team2_items.extend(item["itemID"] for item in items)

    return team1_items, team2_items


# Disable SSL certificate verification
requests.packages.urllib3.disable_warnings()

# Fetch JSON data from the URL
url = "https://127.0.0.1:2999/liveclientdata/playerlist"
response = requests.get(url, verify=False)

# Check if the request was successful
if response.status_code == 200:
    json_data = response.json()
    team1_items, team2_items = collect_items_by_team(json_data)
    print("Team 1 Items:")
    print(team1_items)
    print("Team 2 Items:")
    print(team2_items)
else:
    print("Failed to fetch JSON data. Status code:", response.status_code)


Team 1 Items:
[3071, 6632, 3158, 3091, 3340, 4636, 3041, 4645, 1058, 3020, 1058, 3364, 3020, 3152, 3100, 1058, 1056, 3340, 1055, 3035, 6672, 3006, 6671, 1018, 3363, 3860, 3504, 2424, 6620, 3158, 3364]
Team 2 Items:
[2031, 3074, 3133, 6630, 3158, 1036, 3364, 3051, 6029, 6672, 3111, 3067, 3364, 6653, 3020, 3116, 3135, 1052, 3340, 6671, 6672, 3158, 1055, 3363, 1001, 2065, 3863, 4642, 3108, 3364]


In [68]:
import requests
import json

# Static URL for the items.json data
json_url = "https://cdn.merakianalytics.com/riot/lol/resources/latest/en-US/items.json"

def get_item_price(item_id):
    try:
        response = requests.get(json_url)
        data = json.loads(response.text)

        item_data = data[str(item_id)]
        total_price = item_data["shop"]["prices"]["total"]
        return total_price

    except (requests.RequestException, json.JSONDecodeError, KeyError):
        return None

def calculate_total_item_price(item_ids):
    total_price = 0
    for item_id in item_ids:
        price = get_item_price(item_id)
        if price is not None:
            total_price += price
    return total_price

# Calculate total item price for team 1
team1_total_price = calculate_total_item_price(team1_items)

# Calculate total item price for team 2
team2_total_price = calculate_total_item_price(team2_items)

# Print the results
print(f"Total item price for team 1: {team1_total_price} gold.")
print(f"Total item price for team 2: {team2_total_price} gold.")

Total item price for team 1: 46650 gold.
Total item price for team 2: 39035 gold.


In [69]:
team_1 = {"Gold": 0, "Level": 0, "Minions": 0, "Jungle_minions": 0, "Kills": 0, "Assists": 0, "Deaths": 0, "Plates": 0, "Towers": 0, "Dragons": 0, "Heralds": 0, "Sight_wards": 0, "Control_wards": 0}

In [70]:
team_1["Level"]=order_average
team_1["Minions"]=order_scores["creepScore"]
team_1["Kills"]=order_scores["kills"]
team_1["Assists"]=order_scores["assists"]
team_1["Deaths"]=order_scores["deaths"]
team_1["Gold"]=team1_total_price

In [71]:
team_1

{'Gold': 46650,
 'Level': 14.6,
 'Minions': 790,
 'Jungle_minions': 0,
 'Kills': 34,
 'Assists': 40,
 'Deaths': 9,
 'Plates': 0,
 'Towers': 0,
 'Dragons': 0,
 'Heralds': 0,
 'Sight_wards': 0,
 'Control_wards': 0}